# Estadística descriptiva

La estadística descriptiva corresponde a los métodos numéricos y gráficos que nos permiten describir y visualizar nuestros datos

También se suele llamar análisis exploratorio de datos y es clave para

- Descubrir patrones en nuestros datos
- Verificar preliminarmente nuestras hipótesis sobre los datos
- Detectar outliers (valores fuera de rango) o valores inválidos en nuestros 
- Seleccionar de manera informada los métodos inferenciales que utilizaremos 

entre otros. El objetivo es describir nuestros datos para mejorar nuestro entendimiento de los mismos

En este lección veremos algunas funciones de `matplotlib`, `numpy` y `scipy.stats` para realizar estadística descriptiva de datos

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt

## Histogramas

El histograma es una representación aproximada de la **distribución de probabilidad** de una variable

Asumiendo que tenemos un vector con realizaciones de una variable aleatoria

$$
[x_1, x_2, \ldots, x_N] \sim X
$$

podemos construir un histograma en tres pasos

1. Establecer un rango mínimo y máximo
1. Dividir el rango en **cajones/bines**
1. Medir la frecuencia relativa en cada cajón, es decir la cantidad de $x_i$ que caen en dicho cajón divido $N$


**Ejemplo:** Para construir el siguiente histograma se utilizaron $7$ bines de tamaño $1$ regularmente distribuidos en el rango $[-3, 4]$

<img src="img/stats4.png">


El ejemplo anterior corresponde al caso particular más común donde los cajones son disjuntos y de igual tamaño. En dicho caso los cajones se definen completamente con

- el rango del histograma: Comienzo del primer bin y termino del último bin
- la cantidad de bines: En conjunto con el rango especifican el tamaño de los bines

:::{important}

El tamaño de los bines relativo al rango del histograma controla la resolución del histograma. Por ejemplo para un rango fijo, utilizar bines muy grandes resultará en pérdida de detalle. Por el contrario utilizar bines muy pequeños resultará en un histograma muy ruidoso

:::


In [3]:
df = pd.read_csv('../linalg/data/helados.csv', header=0, index_col=0)
df.columns = ['consumo', 'ingreso', 'precio', 'temperatura']
display(df.head())

,consumo,ingreso,precio,temperatura
1,0.386,78,0.270,41
2,0.374,79,0.282,56
3,0.393,81,0.277,63
4,0.425,80,0.280,68
5,0.406,76,0.272,69



### Interpretación de un histograma

El histograma revela visualmente algunas propiedades y comportamientos de la muestra. Considere los siguientes ejemplos de histogramas de distribuciones univariadas

<img src="img/stats5.png">

El histograma nos permite ver

- Cantidad de modas: Una, dos,  múltiples o ninguna en particular (distribución uniforme)
- Distribución simétrica o asimétrica. En una distribución asimétrica el peso puede estar hacia la cola izquierda o derecha


### Construyendo histogramas con `matplotlib`

Podemos construir y visualizar un histograma con matplotlib usando

```python
plt.hist(x, # datos
         bins=None, # Cantidad de cajas (escalar) o bordes de las cajas (secuencia creciente)
         range=None,  # Valor mínimo y máximo del histograma
         density=False, #dividir por len(x) o no
         ...
        )
```


Para el caso de dos variables se puede hacer un histograma bidimensional (mapa de contornos) con 

```python
plt.hist2d(x, # data eje horizontal
           y, # data eje vertical
           bins=10, # Igual que hist
           range=None, # Arreglo [[xmin, xmax],[ymin,ymax]]
           density=False, #Igual que hist
           ...
          )
```

Las alternativas en la librería `seaborn` serían [`seaborn.distplot`](https://seaborn.pydata.org/generated/seaborn.distplot.html) y [`seaborn.jointplot`](https://seaborn.pydata.org/generated/seaborn.jointplot.html), respectivamente

**Ejemplo**

is the first step in the data analysis process. It allows us to understand the data we are dealing with by describing and summarizing the dataset’s main characteristics, often through visual methods like bar and pie charts, histograms, boxplots, scatterplots, heatmaps, and many more.

Antes de responder preguntas con nuestros datos debemos es conveniente explorarlos

> La estadística descriptiva es un conjunto de técnicas que 

> Para entender la muestra podemos intentar describirla

Considere por ejemplo las siguientes preguntas

- ¿Son mis datos discretos o continuos? ¿Cuál es su rango?
- ¿Cuál es el valor medio de mis datos? ¿Cuál es su disperción?
- ¿Tiene una moda o múltiples modas?
- ¿Existen datos extremos (*outliers*)?

Todas estas preguntas buscan describir la **distribución** subyacente de nuestros datos

## Repaso de fundamentos

**Distribución de probabilidades**

Una distribución de probabilidad es una función que relaciona los valores posibles de una VA con sus respectivas probabilidades de ocurrir

La definición es un poco distinta dependiendo si la variable aleatoria es discreta o continua

- Variable continua: Toma valores de un conjunto no numerable. Ejemplos: Los números reales
- Variable discreta o categórica: Toma valores de un conjunto numerable. Puede ser ordinal, por ejemplo los números enteros. Puede ser nominal, por ejemplo el género o la étnia de las personas

Ambas pueden estar definidas en un cierto rango

**Función de masa de probabilidad**

Si nuestra variable $x\sim X$ es discreta entonces podemos calcular la probabilidad de un evento como

$$
P(X = x) = p_X(x)
$$

donde $p_X(x)$ se conoce como función de masa de probabilidad (fmp)

Para que una fmp sea valida se debe cumplir

- Estar en el rango de una probabilidad: $p_X(x) \in [0, 1]$
- Sumar a 1 en el dominio completo: $\sum_{x\in\mathcal{X}} p_X(x) = 1$    

**Función de densidad de probabilidad**

Si nuestra variable $x\sim X$ es continua entonces debemos definir un rango si queremos estimar su probabilidad

$$
P(a < X < b) = \int_{a}^{b} f_X(x) \,dx = F_X(b) - F_X(a)
$$

donde $f_X(\cdot)$ es la función de densidad de probabilidad (fdp) y $F_X(\cdot)$ es la función de densidad acumulada

Para que una fdp sea valida se debe cumplir

- No-negatividad: $f_X(x) \geq 0$
- Integrar a 1 en el dominio completo: $\int_{x\in\mathcal{X}} f_X(x) \,dx = 1$ 
  
  
**Probabilidad conjunta, marginal y condicional**

Las definiciones anteriores son para fdp/fmp de una variable

Si queremos calcular la probabilidad de que $X=x$ y $Y=y$ entonces necesitamos la distribución conjunta $P(X, Y)$ 

A partir de la conjunta se pueden obtener la probabilidad marginal de $X$ (o de $Y$)

$$
P(X=x) = \sum_{y \in \mathcal{Y}} P(X=x, Y=y) 
$$

que corresponde a sumar (marginalizar) la variable que no nos interesa

Usando la conjunta y las marginales podemos obtener las probabilidades condicionales

$$
P(Y=y|X=x) = \frac{P(X=x, Y=y)}{P(X=x)}
$$

ssi $P(X=x) \neq 0$

De esta forma podemos relacionar variables estilo causa y efecto

**Distribución acumulada empírica**

En general no sabemos a priori la fdp/fmp de los datos 

Podemos calcular probabilidades usando la distribución empírica

Si tengo $n$ datos $X_1, X_2, \ldots, X_n$, su distribución acumulada empírica es

$$
F_n(x) = \frac{1}{n} \sum_{i=1}^n \mathbb{1}(X_i < x)
$$

Donde la función indicadora $\mathbb{1}(X_i<x)$ es igual a $1$ si la $X_i<x$ o $0$ en el caso contrario

Si los datos están ordenados entonces sólo tenemos que buscar en que posición se encuentra $x$ y dividir por $n$

Podemos implementarla en NumPy como

```python
def ECDF(data):
    data = np.sort(data)
    n = len(data)
    def result(x):
        return np.searchsorted(data, x, side='right')/n
    return result
```

Luego si tengo una arreglo de datos $a$ y quiero saber la probabilidad de $b$ 

```python
ECDF(a)(b)
```

**Ejemplo**

## Descriptores de la distribución

Visualizar la distribución nos da mucha información. Pero en algunos casos sólo interesa comprobar la existencia de ciertas características o atributos de la distribución

> Podemos resumir una distribución en función de sus **momentos**

Cada momento describe una propiedad en particular

- Primer momento o media: Describe donde está localizada la distribución
- Segundo momento o varianza: Describe el ancho o dispersión de la distribución
- Tercer momento o simetría (skewness): Indica si la distribución es más pesada hacia la izquierda o hacia la derecha
- Cuarto momento o curtosis (kurtosis): Indica si la distribución es picuda o plana

La siguiente figura esquematiza las influencia de los momentos

<img src="https://images.deepai.org/django-summernote/2019-07-02/4d0a6ff1-8b53-4537-9d0a-919993d10024.png" width="500">

También podemos describir una distribución en función de la

- Moda: Valor más repetido
- Mediana: Valor central
- Diferencias entre percentiles

### Calculando descriptores con Python

Podemos obtener la media y la desviación estándar de un dataframe usando 

```python
df.describe()
```

Podemos obtener los primeros cuatro momentos de un ndarray junto a su largo y rango con

```python
scipy.stats.describe(x)
```

Podemos calcular un momento de cualquier orden con

```python
scipy.stats.moment(x, moment=5)
```

Otros descriptores comunes están implementados en el módulo [scipy.stats](https://docs.scipy.org/doc/scipy/reference/stats.html#summary-statistics)

### Ejemplo

Descriptores para el radio y la textura usando pandas y scipy

In [ ]:
# Descriptores

display(df.describe())

print(scipy.stats.describe(df[["radius1", "texture1"]]))

## Distribuciones comunes 

Existen muchísimas distribuciones de probabilidad asociadas a distintos fenómenos y experimentos. La siguiente tabla presenta las distribuciones más comunmente encontradas 


| Distribución | Fenómeno que representa | Ejemplo |
| --- | --- | --- |
| **Bernoulli** | Evento binario  | Lanzamiento de una moneda |
| **Binomial** | Multiples eventos binarios independientes | |
| **Categórica** | Evento con $k$ valores posibles | Lanzamiento de un dado, Ruleta |
| **Poisson** | Conteo de eventos ocurridos en un período de tiempo | Cantidad de alumnos que llegan entre 9:50 y 10:00| 
| **Exponencial** | Valor continuo positivo | Tiempo de espera entre eventos|
| **Gamma** | Valor continuo positivo | Tiempos de espera hasta que ocurren $n$ eventos|
| **Beta** | Valor continuo en $[0, 1]$ |  Tiempo para completar una tarea, proporciones|
| **Normal/Gaussiana** | Valor continuo ubicado en la vecindad de un valor central| [Demasiados](https://galtonboard.com/probabilityexamplesinlife)|
| **Uniforme** | Valor discreto/continuo acotado a un rango, todos con igual probabilidad de ocurrencia| |

Conocer su forma nos permite reconocerlas cuando estudiamos la densidad empírica de nuestros datos

> Les recomiendo el [zoológico de distribuciones](https://ben18785.shinyapps.io/distribution-zoo/) para ver interactivametne la forma de estas distribuciones

En la próxima lección veremos en mayor detalle como podemos ajustar una distribución teórica a nuestros datos



## Generando muestras aleatorias con NumPy y SciPy

Podemos usar el módulo `np.random` para generar números aleatorios con las siguientes distribuciones

- `randn` : Números reales con distribución normal estándar
- `rand`: Números reales con distribución uniforme en $[0, 1]$
- `randint(low=1, high=10)`: Números enteros con distribución uniforme entre $[0, 10]$

Si necesitamos datos que siguen otra distribución es más recomdable usar el módulo `scipy.stats`. La lista de distribuciones disponible es sumamente extensa y está divida entre tres

- [continua](https://docs.scipy.org/doc/scipy/reference/stats.html#continuous-distributions)
- [multivariada](https://docs.scipy.org/doc/scipy/reference/stats.html#multivariate-distributions)
- [discreta](https://docs.scipy.org/doc/scipy/reference/stats.html#discrete-distributions)

Todas las distribuciones de `scipy.stats` tienen los siguientes métodos 

- `pdf`/`pmf(x)` Retorna la distribución de probabilidad evaluada en $x$
- `cdf(x)` Distribución acumulada evaluada en $x$
- `ppf(p)` Inverso de la distribución acumulada
- `rvs(size=100)` Retorna $100$ muestras a partir de la distribución

**Ejemplo**

Generando y visualizando $1000$ muestras de una distribución Gaussiana (normal) con media 2 y desviación estándar 10

In [ ]:
from scipy.stats import norm
dist = norm(loc=2, scale=10) # Constructor
samples = dist.rvs(size=1000) # Generación
fig, ax = plt.subplots(figsize=(7, 3), tight_layout=True)
ax.hist(samples); # Histograma

La próxima clase veremos en mayor detalle como usar estos y otros métodos

## ¿Por qué la distribución Gaussiana para ser tan común?

<img src="https://thumbs.gfycat.com/AggressiveAromaticBuckeyebutterfly-size_restricted.gif">

El siguiente sitio web muestra algunos ejemplos de la fenómenos que se distribuyen de forma Gaussiana: https://galtonboard.com/probabilityexamplesinlife ¿Por qué esta distribución parece ser tan común? 

Recordemos lo siguiente

**Ley de los grandes números**

Sean $X_1, X_2, \ldots, X_N$ V.A independientes e idénticamente distribuidas (iid) con media $\mu$. Se cumple que su promedio

$$
\bar X = \frac{1}{N} (X_1 + X_2 + \ldots + X_N)
$$

tiende a $\mu$ cuando $N \to \infty$

**Teorema central del límite**

Si $X_1, X_2, \ldots, X_N$ son V.A iid, entonces su promedio de distribuye como

$$
\bar X \sim \mathcal{N}(\mu, \sigma^2/N)
$$

Es decir que 

> El promedio (o suma) **se distribuye normal sin importar la distribución original** de las V.A. 

siempre y cuando las VA sean independientes. ¿Cúantos de los ejemplos del sitio web mencionado corresponden a sumas o promedios?

### Ejercicio

Sea una variable aleatoria que corresponde al lanzamiento de $n$ dados de seis caras. Lance los $n$ dados 1000 veces y muestre con un histograma la distribución del promedio de los $n$ dados. Repita para $n: 1, 2, ..., 15$

¿Qué puede decir de la distribución que observa?

Hint: Puede usar `scipy.stats.multinomial` para modelar el resultado de lanzar $n$ dados

**Solución paso a paso con comentarios**

In [ ]:
YouTubeVideo_formato('2QwMBZAlvc4')